In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numba as nb
import scipy as sp
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from pathlib import Path
import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf

In [2]:
# Code Needs: (eventually)
# - ROICaT index alignment tools (will wait until I have data processed and loaded from multiple sessions)

In [3]:
# Immediate next steps for programming work:

# - red cell processing functions
# - database development


In [10]:
# vrExperiment registration and preprocessing 
mouseName = 'ATL022'
dateString = '2023-03-24'
session = '701'
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In session: ATL022/2023-03-24/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
Facecam preprocessing has not been coded yet!
Red cell processing has not been coded yet!
C:\Users\andrew\Documents\localData\ATL022\2023-03-24\701


In [11]:
mouseName = 'ATL022'
dateString = '2023-03-24'
session = '701'
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [111]:
%%time
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=5) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 13 s
Wall time: 3.06 s
